This notebook creates a csv file in the necessary format.

# Changes to make
- make sure the code is as general as possible


# Possible improvements
- minimize NaNs in bridges (especially name/year/type)

# Changes made
- filter for N-roads only (still over 25km long)
- there is only one link between bridges and intersections (and combination thereof)
- add number of trucks that are created and (U) that are destroyed at each sourcesink


In [1]:
import pandas as pd
from scipy.spatial import distance
import os
import math

# Load raw datafiles into DataFrames
roads = pd.read_csv("../data/raw/_roads3.csv")
bridges = pd.read_excel('../data/raw/BMMS_overview.xlsx', engine='openpyxl')

In [2]:
#--- Data processing input

# Specify main roads to find side roads to
mainroad = ['N1', 'N2']

# Specify road types to include
road_type = "N"

# Specify required length of the road to include it
length_required = 25

In [3]:
#--- Identifying side roads of N1 and N2

# Only keep the main roads we are interested in
df = roads.loc[(roads["road"].isin(mainroad))]

# List of all roads
Roads = roads['road'].unique().tolist()

# List of all crossroads and side roads (in ugly format)
names = df['name'].tolist()

# Find roads that appear in the crossroads / sideroads 
side_roads = [road for road in Roads if any(road in name for name in names)]

In [4]:
#--- Filtering for road type and required length

# Take all side roads
sideroads_df = roads[roads['road'].isin(side_roads)]

# Filter for those over x km long (aka LRPE had chainage >x)
sideroads_ends = sideroads_df.loc[(sideroads_df["lrp"] == "LRPE") & (sideroads_df["chainage"] > length_required)]
sideroads_tokeep = sideroads_ends['road'].tolist()

# Apply filter
sideroads_df = sideroads_df.loc[(sideroads_df['road'].isin(sideroads_tokeep))]

# Filter for the right type of road
sideroads_tokeep = [s for s in sideroads_tokeep if road_type in s]

# Apply filter
sideroads_df = sideroads_df.loc[(sideroads_df['road'].isin(sideroads_tokeep))]

print("roads we consider in this assignment", sideroads_df['road'].unique())

roads we consider in this assignment ['N1' 'N102' 'N104' 'N2' 'N204' 'N207' 'N208']


In [5]:
# Show the intermediate dataset
sideroads_df.head()

,road,chainage,lrp,lat,lon,gap,type,name
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...
1,N1,0.814,LRPSa,23.702917,90.450417,NaN,Culvert,Box Culvert
2,N1,0.822,LRPSb,23.702778,90.450472,NaN,CrossRoad,Intersection with Z1101
3,N1,1.000,LRP001,23.702139,90.451972,NaN,KmPost,Km post missing
4,N1,2.000,LRP002,23.697889,90.460583,NaN,KmPost,Km post missing


In [6]:
#--- Adding a length to each road segment

# Create the length column
sideroads_df["length"] = sideroads_df["chainage"]*1000 
sideroads_df = sideroads_df.reset_index(drop=True)

# Change the chainage to a length
for i in range((len(sideroads_df)-1),0,-1):
    if sideroads_df["road"][i] == sideroads_df["road"][i-1]:
        sideroads_df.loc[i,"length"] = sideroads_df.loc[i,"length"]-sideroads_df.loc[i-1,"length"]
sideroads_df.head()

,road,chainage,lrp,lat,lon,gap,type,name,length
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...,0.0
1,N1,0.814,LRPSa,23.702917,90.450417,NaN,Culvert,Box Culvert,814.0
2,N1,0.822,LRPSb,23.702778,90.450472,NaN,CrossRoad,Intersection with Z1101,8.0
3,N1,1.000,LRP001,23.702139,90.451972,NaN,KmPost,Km post missing,178.0
4,N1,2.000,LRP002,23.697889,90.460583,NaN,KmPost,Km post missing,1000.0


In [7]:
#--- Moving on to bridges!

# Filter for relevant columns and relevant roads
bridges_relevant = bridges[["road", "LRPName", "condition", "length", "chainage", "lat", "lon", 'name', 'km','constructionYear']]
bridges_relevant = bridges_relevant.loc[bridges['road'].isin(sideroads_tokeep)]
bridges_relevant = bridges_relevant.reset_index(drop = True)

bridgestemp = bridges_relevant # useful in a second

# Only keep right side of each bridge
for i in range(1,len(bridges_relevant)):
    if len(str(bridges_relevant["name"][i])) > 4:
        if bridges_relevant["name"][i][-2:] == 'L)' or bridges_relevant["name"][i][-4:] == 'eft)' or bridges_relevant["name"][i][-3:] == 'L )' or bridges_relevant["name"][i][-4:] == 'EFT)':
            bridgestemp = bridgestemp.drop(i,axis = 0)

# Delete depulicates by removing older information
# assumption: no 2 lrps in the same road have the exact same km
bridges_relevant = bridgestemp \
    .sort_values(by=['road','km','constructionYear'], ascending=False) \
    .drop_duplicates(subset=['road', 'km'], keep='first')
bridges_relevant.head()

,road,LRPName,condition,length,chainage,lat,lon,name,km,constructionYear
1471,N208,LRP057b,B,5.6,57.089,24.878083,91.875333,KADAM TALI BOX CULVERT,57.089,1985.0
1127,N208,LRP055a,A,3.0,55.040,24.858606,91.885036,SIB BARI BOX CULVERT,55.040,1985.0
1126,N208,LRP052a,A,32.2,51.311,24.831194,91.899334,JALKER KANDI,51.311,1965.0
1125,N208,LRP051a,A,3.0,50.390,24.824334,91.901492,NAIKHAI BOX CULVERT,50.390,1975.0
1123,N208,LRP048a,A,20.8,47.380,24.800161,91.912263,MOGLA BAZER,47.380,1985.0


In [8]:
#--- Bringing relevant roads and bridges together 

# Prepare merge
bridges_relevant = bridges_relevant.rename(columns={"LRPName": "lrp"})

# Merge
merged = pd.merge(sideroads_df, bridges_relevant, how="outer", on=["road", "lrp"])
merged = merged.reset_index(drop = True)

# Add model_type
merged["model_type"] = merged["lrp"].apply(lambda x: "sourcesink" if x == "LRPS" else ("sourcesink" if x == "LRPE" else "link"))
merged.loc[merged["condition"].notnull(), "model_type"] = "bridge"

# Fill in missing data
merged["chainage_x"] = merged["chainage_x"].fillna(value=merged["chainage_y"])
merged["lat_x"] = merged["lat_x"].fillna(value=merged["lat_y"])
merged["lon_x"] = merged["lon_x"].fillna(value=merged["lon_y"])
merged["name_y"] = merged["name_y"].fillna(value=merged["name_x"])
merged["length_x"] = merged["length_x"].fillna(value=merged["length_y"])

# Keep and rename useful columns only
merged = merged.sort_values(by=['road','chainage_x'],ascending = True)
col_tokeep = ["road", "model_type", "lrp", "name_y", "lat_x", "lon_x", "length_x", "condition","type",'chainage_x']
merged = merged[col_tokeep]
merged = merged.rename(columns={"name_y": "name",'chainage_x':'chainage',"lat_x" : "lat", "lon_x" : "lon", "length_x" : "length"})
merged = merged.reset_index(drop = True)

# Add ids
merged["id"] =  range(1000000, len(merged) +1000000 )
cols = ["road", "model_type", "lrp", "name", "lat", "lon", "length", "condition","type","id", 'chainage']
merged = merged[cols]

In [9]:
#--- Defining the intersections

# subset the dataframe to only the rows that indicate an intersection
cross_sideroads = merged.loc[(merged['type'].str.contains("CrossRoad"))| (merged['type'].str.contains("SideRoad"))]
cross_sideroads = cross_sideroads.reset_index()

roads_done = []
# iterate over all road rows that are crossroads or sideroads
for i,crossrow in cross_sideroads.iterrows():
    # check if any of those roads indicate in their name that they intersect with a road that crosses the N1 and/or N2
    for j in sideroads_tokeep:
        if j in str(crossrow["name"]) and j != crossrow["road"]:
            # if you are not trying one intersection that you already did the other way around
            if i not in roads_done:
                # iterate over all points in that intersecting road to see if any of those indicate to be an intersection with the road at hand
                # since a road can cross another road on multiple occasions, we only look as far as 1 lat and 1 lon
                for k,roadrow in merged[merged['road'] == j].iterrows():
                    if crossrow["name"] in roadrow["name"] and (roadrow["lat"] - crossrow["lat"] < 1) and (roadrow["lon"] - crossrow["lon"] < 1) and crossrow["road"] != roadrow["road"]:
                        # if this finds the intersection point, make its id, lat and lon the same and make the model_type "intersection"
                        merged.iloc[k,4] = cross_sideroads.iloc[i,5] 
                        merged.iloc[k,5] = cross_sideroads.iloc[i,6]
                        for l, newestrow in merged.loc[merged["id"] == roadrow["id"]].iterrows():
                            merged.iloc[l,9] = cross_sideroads.iloc[i,10]                          
                            merged.iloc[l,1] = "intersection"                      
                        merged.iloc[cross_sideroads.iloc[i,0],1] = "intersection"
                        #print("made indersection here1:",merged.iloc[k,:],merged.iloc[cross_sideroads.iloc[i,0],1])
                        # save the fact that you have handled this intersection
                        roads_done.append(i)
                        break
                # if you didnt find the intersecting road through the name, assign the closest road point as the intersection
                # assumption: projection is so small that it will not distort the distance to much so we can use euclidean distance to determine the closest point
                else:
                    closestrow = merged.iloc[[0]]
                    closestindex = 0
                    closestdistance = 1000
                    p1 = (crossrow["lat"],crossrow["lon"])
                    # calculate for each road point the distance to the intersection, save it if it is closer than what you found before
                    for k,roadrow in merged[merged['road'] == j].iterrows():
                        p2 = (roadrow["lat"],roadrow["lon"])
                        if distance.euclidean(p1, p2) < closestdistance:
                            closestrow = roadrow
                            closestindex = k
                            closestdistance = distance.euclidean(p1, p2)
                    # of the closest point, make the id, lat and lon the same and make the model_type "intersection"
                    merged.iloc[closestindex,9] = cross_sideroads.iloc[i,10]
                    merged.iloc[closestindex,4] = cross_sideroads.iloc[i,5]
                    merged.iloc[closestindex,5] = cross_sideroads.iloc[i,6]
                    merged.iloc[closestindex,1] = "intersection"
                    merged.iloc[cross_sideroads.iloc[i,0],1] = "intersection"
                    #print("made indersection here2:",merged.iloc[k,:],"other roadpoint:",merged.iloc[cross_sideroads.iloc[i,0],:])
                    # save the fact that you have handled this intersection
                    roads_done.append(i)    
    

In [10]:
# as the above code does not find all intersections, here is a more crude way to find them anyhow based on the closeness of the points in different roads
# warning: takes a very long time to run
for i, row in merged.iterrows():
    for j in range(i,len(merged)):
        if merged['model_type'][j] != 'bridge':
            if row['id'] != merged['id'][j] and merged['model_type'][j] != 'intersection' and row['road'] != merged['road'][j] and distance.euclidean((row['lat'],row['lon']), (merged['lat'][j],merged['lon'][j])) < 0.001:
                merged.iloc[j,9] = merged.iloc[i,9]
                merged.iloc[j,4] = merged.iloc[i,4]
                merged.iloc[j,5] = merged.iloc[i,5]
                merged.iloc[j,1] = "intersection"
                merged.iloc[i,1] = "intersection"
                print('new intersection made at id',row['id'],'between', row['road'], 'and', merged['road'][j])
                break

new intersection made at id 1001932 between N1 and N2
new intersection made at id 1000374 between N1 and N104
new intersection made at id 1002257 between N2 and N204
new intersection made at id 1002595 between N2 and N207
new intersection made at id 1003113 between N207 and N208


In [11]:
# In our case, only one intersection is missing for the Road N104. 

#23.009555600, 91.3813604 on N1
#23.0095278 , 91.3814438 on N104

# index of the soon-to-be intersection on N104
#d = merged.loc[(merged["lat"] == 23.0095278) & (merged["lon"] == 91.3814438)]

# making it an intersection
#d.model_type = "intersection"

# putting it back on N104 instead of the old link
#merged.loc[(merged["lat"] == 23.0095278) & (merged["lon"] == 91.3814438)] = d

# index of bridge on N1 that is also an intersection
#index_bridge = merged.loc[(merged["lat"] == 23.009555600000002) & (merged["lon"] == 91.3813604)].index 

# inserting a row below that bridge with the intersection
#d.road = 'N1'
#line = pd.DataFrame(data = d)
#line.index = index_bridge + [0.5] 
#merged = merged.append(line, ignore_index=False)
#merged = merged.sort_index().reset_index(drop=True)


In [12]:
#--- Adapting to the newest csv guidelines

# Adapting names
#move bridge names to a new column 
merged["bridge_name"] = merged["name"].loc[merged['model_type'] == "bridge"]
#delete names for everything and replace that of SourceSinks according to convention
i = 1 # useful in a second
for index, row in merged.iterrows():
    if not row['model_type'] == "sourcesink":
        merged["name"][index] = ""
        
    elif row['model_type'] == "sourcesink":
        merged["name"][index] = "SoSi" + str(i)
        merged["condition"][index] = ""
        i += 1

# Put columns in right order
merged = merged[["road", "id", "model_type", "condition", "name", "lat", "lon", "length", "bridge_name","chainage"]]
merged.head()

<ipython-input-12-5038fc133b90>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["name"][index] = "SoSi" + str(i)
<ipython-input-12-5038fc133b90>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["condition"][index] = ""
<ipython-input-12-5038fc133b90>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["name"][index] = ""


,road,id,model_type,condition,name,lat,lon,length,bridge_name,chainage
0,N1,1000000,sourcesink,,SoSi1,23.706028,90.443333,0.0,NaN,0.000
1,N1,1000001,link,NaN,,23.702917,90.450417,814.0,NaN,0.814
2,N1,1000002,link,NaN,,23.702778,90.450472,8.0,NaN,0.822
3,N1,1000003,link,NaN,,23.702139,90.451972,178.0,NaN,1.000
4,N1,1000004,bridge,A,,23.698739,90.458861,11.3,.,1.800


In [13]:
for i,row in merged.iterrows():
    if row['bridge_name'] == '.':
        merged.iloc[i,8] = 'bridge at id '+ str(row['id'])

In [14]:
# Rename dataset to a more intuitive name
Roads_df = merged

In [15]:
# create two new columns, one for the ingoing traffic and one for the outgoing traffic

Roads_df['in'] = pd.Series()
Roads_df['out'] = pd.Series()

<ipython-input-15-c99214e4f1c1>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Roads_df['in'] = pd.Series()
<ipython-input-15-c99214e4f1c1>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Roads_df['out'] = pd.Series()


In [16]:
# For each sourcesink, find the closest point in the traffic data by comparing chainages.

for index, row in Roads_df.iterrows():
    if row['model_type'] == "sourcesink":
        path = '../data/raw/traffic/'+row['road']+'.traffic.htm'
        trafficdata = pd.read_html(path)[2]
        trafficdata = trafficdata.iloc[4:-3,[0,1,4,5,25]]
        trafficdata.columns = trafficdata.iloc[0]
        trafficdata = trafficdata[2:]
        trafficdata["Start location"] = trafficdata["Start location"].astype(float)*1000
        trafficdata['Traffic'] = trafficdata['Traffic'].astype(float)
        closestrow = trafficdata.iloc[[0]]
        closestdistance = 1000
        # For some html files, it can occur that traffic is saved separately for ingoing traffic and for outgoing traffic
        # If that is the case, we save those separately, otherwise, we divide the traffic equally over the 'in' and 'out' columns
        closestrowright = trafficdata.iloc[[0]]
        twowaydata = False        
        for i,r in trafficdata.iterrows():
            if abs(row['chainage']-r['Start location']) == closestdistance:
                closestrowright = r
                twowaydata = True
            elif abs(row['chainage']-r['Start location']) < closestdistance:
                closestrow = r
                closestdistance = abs(row['chainage']-r['Start location'])
        if twowaydata == True:
            Roads_df.iloc[index,10] = closestrowright['Traffic']
            Roads_df.iloc[index,11] = closestrow['Traffic']
        else:
            Roads_df.iloc[index,10] = closestrow['Traffic']/2
            Roads_df.iloc[index,11] = closestrow['Traffic']/2

In [17]:
# Show it worked

Roads_df.head()

,road,id,model_type,condition,name,lat,lon,length,bridge_name,chainage,in,out
0,N1,1000000,sourcesink,,SoSi1,23.706028,90.443333,0.0,NaN,0.000,21537.0,19357.0
1,N1,1000001,link,NaN,,23.702917,90.450417,814.0,NaN,0.814,NaN,NaN
2,N1,1000002,link,NaN,,23.702778,90.450472,8.0,NaN,0.822,NaN,NaN
3,N1,1000003,link,NaN,,23.702139,90.451972,178.0,NaN,1.000,NaN,NaN
4,N1,1000004,bridge,A,,23.698739,90.458861,11.3,bridge at id 1000004,1.800,NaN,NaN


In [18]:
# Save the traffic as a fraction/probability of trucks generation

total_in = Roads_df['in'].sum()
total_out = Roads_df['out'].sum()
for index, row in Roads_df.iterrows():
    if row['model_type'] == "sourcesink":
        Roads_df.iloc[index,10] = row['in']/total_in
        Roads_df.iloc[index,11] = row['out']/total_out

In [19]:
# Show it worked

print('sum of in column is', Roads_df['in'].sum())
print('sum of out column is', Roads_df['out'].sum())
Roads_df.loc[Roads_df['in'].notnull()] 

sum of in column is 1.0
sum of out column is 1.0


,road,id,model_type,condition,name,lat,lon,length,bridge_name,chainage,in,out
0,N1,1000000,sourcesink,,SoSi1,23.706028,90.443333,0.0,NaN,0.000,0.319244,0.286930
1554,N1,1001554,sourcesink,,SoSi2,20.862917,92.298083,130.0,NaN,462.254,0.286930,0.319244
1782,N102,1001782,sourcesink,,SoSi3,24.050611,91.114667,549.0,NaN,82.682,0.071951,0.071951
1783,N104,1001783,sourcesink,,SoSi4,23.009667,91.399416,0.0,NaN,0.000,0.069557,0.069557
1913,N104,1001913,sourcesink,,SoSi5,22.825749,91.101444,120.0,NaN,49.630,0.069557,0.069557
2876,N2,1002876,sourcesink,,SoSi6,25.157056,92.017638,256.0,NaN,286.516,0.182761,0.182761


In [20]:
# Drop the chainage column

Roads_df = Roads_df[["road", "id", "model_type", "condition", "name", "lat", "lon", "length", "bridge_name","in","out"]]

In [21]:
d = Roads_df.iloc[0,:]
Sparse_df = pd.DataFrame(data = d)
Sparse_df = Sparse_df.transpose()

In [22]:
# Merge al consecutive links together to one big link

chainage_build_up = 0
number_of_links = 0

for index, row in Roads_df.iterrows():
    if row["model_type"] == 'link':
        chainage_build_up += row['length']
        number_of_links += 1
    elif number_of_links > 0:
        if number_of_links == 1:
            Sparse_df = Sparse_df.append(Roads_df.iloc[index-1,:])
        else:
            Sparse_df = Sparse_df.append(Roads_df.iloc[index-math.floor(number_of_links/2)-1,:])
            Sparse_df.iloc[-1,7] = chainage_build_up
        chainage_build_up = 0
        number_of_links = 0
        Sparse_df = Sparse_df.append(row)
    elif index > 0:
        Sparse_df = Sparse_df.append(row)

In [23]:
# Show it worked

Sparse_df.head()

,road,id,model_type,condition,name,lat,lon,length,bridge_name,in,out
0,N1,1000000,sourcesink,,SoSi1,23.706,90.4433,0,NaN,0.319244,0.28693
2,N1,1000002,link,NaN,,23.7028,90.4505,1000,NaN,NaN,NaN
4,N1,1000004,bridge,A,,23.6987,90.4589,11.3,bridge at id 1000004,NaN,NaN
7,N1,1000007,link,NaN,,23.6938,90.4691,3175,NaN,NaN,NaN
10,N1,1000010,bridge,A,,23.6947,90.4878,6.6,bridge at id 1000010,NaN,NaN


In [24]:
Sparse_df = Sparse_df.reset_index()

In [25]:
# Save to csv 

Sparse_df.to_csv("../data/processed/N1_N2_plus_sideroads.csv", index=None)